# Load and save data

In [1]:
import datetime
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
#import numpy as np
import calendar

In [2]:
# Load data
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data = pd.read_html(url)

# Split data
sp500 = data[0]
changes = data[1]

In [3]:
# Work on sp500 DataFrame
columns = ['Symbol',
           'Security',
           'GICS Sector',
           'GICS Sub-Industry',
           'Date added']
sp500 = sp500[columns]
display(sp500.head())
print(f'Number of rows: {sp500.shape[0]}\nNumber of columns: {sp500.shape[1]}')

,Symbol,Security,GICS Sector,GICS Sub-Industry,Date added
0,MMM,3M,Industrials,Industrial Conglomerates,1957-03-04
1,AOS,A. O. Smith,Industrials,Building Products,2017-07-26
2,ABT,Abbott,Health Care,Health Care Equipment,1957-03-04
3,ABBV,AbbVie,Health Care,Biotechnology,2012-12-31
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,2011-07-06


Number of rows: 503
Number of columns: 5


In [4]:
# Check null values
for col in columns:
    count = sp500[col].isnull().sum()
    print(f'{count} null values in column "{col}"')    

0 null values in column "Symbol"
0 null values in column "Security"
0 null values in column "GICS Sector"
0 null values in column "GICS Sub-Industry"
0 null values in column "Date added"


In [5]:
# Check correct format of date column
n_incorrect = sp500.shape[0] - (sp500['Date added'].str.fullmatch('^\d{4}-\d{2}-\d{2}')).sum().item()
print(f'Number of incorrect dates: {n_incorrect}')

Number of incorrect dates: 0


In [6]:
# Save dataset
today = datetime.date.today()
year = today.year
month = str(today.month).zfill(2)
day = str(today.day).zfill(2)
filename = f'data/sp500_companies_{year}_{month}_{day}.xlsx'
sp500.to_excel(filename, index=False)

In [7]:
# Work on changes DataFrame
display(changes.head())
print(f'Number of rows: {changes.shape[0]}\nNumber of columns: {changes.shape[1]}')

Date  Added              Removed                             \
            Date Ticker     Security  Ticker                   Security   
0  June 24, 2024    KKR          KKR     RHI                Robert Half   
1  June 24, 2024   CRWD  CrowdStrike     CMA                   Comerica   
2  June 24, 2024   GDDY      GoDaddy    ILMN             Illumina, Inc.   
3    May 8, 2024    VST       Vistra     PXD  Pioneer Natural Resources   
4  April 3, 2024    NaN          NaN    XRAY            Dentsply Sirona   

                                              Reason  
                                              Reason  
0                   Market capitalization change.[4]  
1                   Market capitalization change.[4]  
2                   Market capitalization change.[4]  
3  S&P 500 and S&P 100 constituent ExxonMobil acq...  
4                   Market capitalization change.[6]

Number of rows: 348
Number of columns: 6


In [8]:
# Modify first column
month_dict = dict((v, k) for k, v in enumerate(calendar.month_name))
def convert_date(date: list):
    assert len(date) == 3
    month, day, year = date[0], date[1], date[2]
    month = str(month_dict[month.lower().title()]).zfill(2)
    day = str(day).zfill(2)
    result = f'{year}-{month}-{day}'
    assert len(result) == 10
    return result

In [9]:
date = changes[('Date', 'Date')]
date = date.str.replace(',', '').str.split()
for i in range(len(date)):
    date[i] = convert_date(date[i])

In [10]:
# Create new dataframe
changes_new = pd.DataFrame({
    'Date': date,
    'Added Ticker': changes[('Added', 'Ticker')],
    'Added Security': changes[('Added', 'Security')],
    'Removed Ticker': changes[('Removed', 'Ticker')],
    'Removed Security': changes[('Removed', 'Security')],
})

In [11]:
display(changes_new)

,Date,Added Ticker,Added Security,Removed Ticker,Removed Security
0,2024-06-24,KKR,KKR,RHI,Robert Half
1,2024-06-24,CRWD,CrowdStrike,CMA,Comerica
2,2024-06-24,GDDY,GoDaddy,ILMN,"Illumina, Inc."
3,2024-05-08,VST,Vistra,PXD,Pioneer Natural Resources
4,2024-04-03,NaN,NaN,XRAY,Dentsply Sirona
...,...,...,...,...,...
343,1999-06-09,WLP,Wellpoint,HPH,Harnischfeger Industries
344,1998-12-11,FSR,Firstar,LDW,Amoco
345,1998-12-11,CCL,Carnival Corp.,GRN,General Re
346,1998-12-11,CPWR,Compuware,SUN,SunAmerica


In [12]:
# Save dataset
today = datetime.date.today()
year = today.year
month = str(today.month).zfill(2)
day = str(today.day).zfill(2)
filename = f'data/sp500_variations_{year}_{month}_{day}.xlsx'
changes_new.to_excel(filename, index=False)